In [ ]:
import numpy as np
import scipy
import matplotlib.pyplot as plt 
import matplotlib.image as mpimg
from scipy.ndimage import label
import scipy.ndimage.measurements
import os
import pandas as pd

In [ ]:
def rgb2gray(rgb):
    return np.dot(rgb[...,:3], [0.2989, 0.5870, 0.1140])

In [ ]:
def plotblackwhite(image):
    imgplt=plt.imshow(image, cmap="gray")
    plt.colorbar()
    plt.show()
    #plt.savefig("soho1264_070205_1742_2_BW_RGB.png", dpi=1000)

In [ ]:
def plotbwr(diff):
    imgplot=plt.imshow(diff, cmap="bwr")
    plt.colorbar()
    imgplot.set_clim(-255,255)
    plt.show()
    #plt.savefig("soho1264_070205_1718_DIFF_1742_3_BWR.png", dpi=1000)

In [ ]:
def plotblodges(l):
    imgplt=plt.imshow(l)
    imgplt.set_clim(0,11)
    imgplt.set_cmap("gray")
    plt.colorbar()
    plt.show()

In [ ]:
filename1 = "full_soho1264_070205_1718.gif"
filename2 = "full_soho1264_070205_1742.gif"

In [ ]:
def CommonPhenomenaFinder(filename1, filename2): 
    
    rgb_image1=mpimg.imread(filename1)
    image1 = rgb2gray(rgb_image1)

    rgb_image2=mpimg.imread(filename2)
    image2 = rgb2gray(rgb_image2)
    
    #plotblackwhite(image1)
    #plotblackwhite(image2)

    diff=np.subtract(image2,image1) # subtract image1 from image2

    #plotbwr(diff)

    x=diff[824:924,50:150].astype(int) #zoom in
    d1=np.where(x<-50, x, 0)   # identify bright blue pixels
    d2=np.where(x>50, x, 0) # identify bright red pixels

    #plotbwr(d1+d2)

    l1, n1 = label(d1, scipy.ones((3,3)))
    l2, n2 = label(d2, scipy.ones((3,3)))

    #plotblodges(l1)
    #plotblodges(l2)

    pairs=list()

    centres1=scipy.ndimage.measurements.center_of_mass(d1,l1,range(1,n1+1))
    centres2=scipy.ndimage.measurements.center_of_mass(d2,l2,range(1,n2+1))

    for c1 in centres1:
       for c2 in centres2:
          if (c1[0]-c2[0])**2 + (c1[1]-c2[1])**2 < 10*10: # if blob is inside search radius of 10 pixels
             pairs.append((c1,c2)) # add centres to pairs list
    #print(len(pairs))

    imgplt=plt.imshow(d1+d2) # display image
    imgplt.set_clim(-255,255)
    imgplt.set_cmap("bwr")   # using gray colourscale
    plt.colorbar()  # adds a colourbar
    xs = [x[0][1] for x in pairs]
    ys = [x[0][0] for x in pairs]
    plt.scatter(xs, ys, marker='x', color='blue')
    xs_ = [x[1][1] for x in pairs]
    ys_ = [x[1][0] for x in pairs]
    plt.scatter(xs_, ys_, marker='x', color='red')
    plt.show()

In [ ]:
CommonPhenomenaFinder(filename1, filename2)